## Testing LLM Factory

In [ ]:
from src.common.llm import LLM
from src.common.types import LLMConfig

llm_factory = LLM(provider="openai", config={"model": "gpt-4o-mini", "reasoning": "thinking"})

llm = llm_factory.get_llm()
llm.invoke("Hello, how are you?")

AIMessage(content="Hello! I'm just a digital assistant so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 13, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bv2sCwaCsBJWcUG1dYbRUedmJrMes', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e52e57ab-190f-417d-9989-46416008e517-0', usage_metadata={'input_tokens': 13, 'output_tokens': 36, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from src.agent.graph import build_graph
import time

@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    time.sleep(10)
    return a + b

@tool
def sub(a: int, b: int) -> int:
    """Subtract two numbers"""
    time.sleep(10)
    return a - b

@tool
def mul(a: int, b: int) -> int:
    """Multiply two numbers"""  
    time.sleep(10)
    return a * b

tools = [add, sub, mul]

graph = build_graph(tools)

response = graph.invoke(
    {
        "messages": [
            HumanMessage(content="What is 2 + 2?")
        ],
        "llm_config": {
            "model": "gpt-4o-mini",
        },
        "llm_provider": "openai"
    }
)

for message in response["messages"]:
    print(message)
    print("-"*100)

Tool call: {'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'call_5Xp3hXFvAEFmAfn6tJkv6Y2L', 'type': 'tool_call'}
Tool result: 4
content='What is 2 + 2?' additional_kwargs={} response_metadata={} id='36912585-44d4-4d47-9e5b-b1eab8860291'
----------------------------------------------------------------------------------------------------
content='' additional_kwargs={'tool_calls': [{'id': 'call_5Xp3hXFvAEFmAfn6tJkv6Y2L', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 108, 'total_tokens': 125, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bv47LUMAtJtjF14AZddG7cJFPS9yh', 'service_tier': 'default', 'finish_reason': 'tool_calls', 

In [3]:
from langgraph import graph
from src.agent.graph import build_graph

graph = build_graph(tools)

for type, chunk in graph.stream(
        {
            "messages": [
                HumanMessage(content="What is 2 + 2?")
            ],
            "llm_config": {
                "model": "gpt-4o-mini",
            },
            "llm_provider": "openai"
        },
        stream_mode=["custom", "messages"]
    ):

    print(type)
    print(chunk)
    print("-"*100)

messages
(AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_CeoIIg6NNWYBGpTsDZd6ohMM', 'function': {'arguments': '', 'name': 'add'}, 'type': 'function'}]}, response_metadata={}, id='run--adde8cd1-ffeb-48a3-8156-9f7671981229', tool_calls=[{'name': 'add', 'args': {}, 'id': 'call_CeoIIg6NNWYBGpTsDZd6ohMM', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'add', 'args': '', 'id': 'call_CeoIIg6NNWYBGpTsDZd6ohMM', 'index': 0, 'type': 'tool_call_chunk'}]), {'langgraph_step': 1, 'langgraph_node': 'chatbot', 'langgraph_triggers': ('branch:to:chatbot',), 'langgraph_path': ('__pregel_pull', 'chatbot'), 'langgraph_checkpoint_ns': 'chatbot:e608e6f7-f029-29d8-e4bb-5739115fdb66', 'checkpoint_ns': 'chatbot:e608e6f7-f029-29d8-e4bb-5739115fdb66', 'ls_provider': 'openai', 'ls_model_name': 'gpt-3.5-turbo', 'ls_model_type': 'chat', 'ls_temperature': None})
----------------------------------------------------------------------------------------------------
messages
(AIMe